https://mp.weixin.qq.com/s/KqZdITK-i8hSm4LO2F9B4Q

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

In [3]:
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [5]:
train_loadr = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('../../dataset/files/',train=True,download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize((0.1307,),(0.3081,))
                               ])
                              ),batch_size=batch_size_train,shuffle=True
)

Using downloaded and verified file: ../../dataset/files/MNIST\raw\train-images-idx3-ubyte.gz
Extracting ../../dataset/files/MNIST\raw\train-images-idx3-ubyte.gz to ../../dataset/files/MNIST\raw



0it [00:00, ?it/s]

Using downloaded and verified file: ../../dataset/files/MNIST\raw\train-labels-idx1-ubyte.gz
Extracting ../../dataset/files/MNIST\raw\train-labels-idx1-ubyte.gz to ../../dataset/files/MNIST\raw



 78%|████████████████████████████████████████████████████▋               | 1277952/1648877 [00:01<00:00, 620428.13it/s]
1654784it [00:02, 787385.43it/s]                                                                                       

0it [00:00, ?it/s]

Extracting ../../dataset/files/MNIST\raw\t10k-images-idx3-ubyte.gz to ../../dataset/files/MNIST\raw



  0%|                                                                                         | 0/4542 [00:42<?, ?it/s]

Extracting ../../dataset/files/MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../dataset/files/MNIST\raw
Processing...
Done!


In [6]:
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../../dataset/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [10]:
examples = enumerate(test_loader)

In [12]:
batch_idx,(example_data,example_targets) = next(examples)

In [13]:
example_data.shape

torch.Size([1000, 1, 28, 28])

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size=5)
        self.conv2 = nn.Conv2d(10,20,kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)